In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import numpy as np
import os
import matplotlib.pyplot as plt


In [2]:
INIT_LR = 1e-4
EPOCHS = 10
BS = 32


In [3]:
#Load Dataset
dataset = r"C:\Users\khavy\Downloads\ThermalDataset\Thermal_images"

data = []
labels = []

imagePaths = list(paths.list_images(dataset))
print("Total images:", len(imagePaths))

Total images: 6340


In [4]:
#Loop Through Images
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]

    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)

    data.append(image)
    labels.append(label)


In [5]:
#Convert Labels to Numbers
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = np.array(labels)

data = np.array(data, dtype="float32")


In [6]:
#Train/Test Split
(trainX, testX, trainY, testY) = train_test_split(
    data, labels,
    test_size=0.2,
    stratify=labels,
    random_state=42
)


In [7]:
#Data Augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)


In [8]:
#Load MobileNetV2
baseModel = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_tensor=Input(shape=(224, 224, 3))
)

In [9]:
#Build Head
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten()(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)


In [10]:
#Freeze Base Model
for layer in baseModel.layers:
    layer.trainable = False

#Compile
opt = Adam(learning_rate=INIT_LR)

model.compile(
    loss="binary_crossentropy",
    optimizer=opt,
    metrics=["accuracy"]
)


In [11]:
#Train
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    epochs=EPOCHS
)

Epoch 1/10


158/158 [==============================] - 191s 1s/step - loss: 0.0182 - accuracy: 0.9950 - val_loss: 2.5137e-04 - val_accuracy: 1.0000
Epoch 2/10
158/158 [==============================] - 263s 2s/step - loss: 2.1604e-04 - accuracy: 1.0000 - val_loss: 5.5047e-05 - val_accuracy: 1.0000
Epoch 3/10
158/158 [==============================] - 191s 1s/step - loss: 7.6055e-05 - accuracy: 1.0000 - val_loss: 2.4129e-05 - val_accuracy: 1.0000
Epoch 4/10
158/158 [==============================] - 189s 1s/step - loss: 6.3361e-05 - accuracy: 1.0000 - val_loss: 1.2485e-05 - val_accuracy: 1.0000
Epoch 5/10
158/158 [==============================] - 188s 1s/step - loss: 4.4659e-05 - accuracy: 1.0000 - val_loss: 6.9794e-06 - val_accuracy: 1.0000
Epoch 6/10
158/158 [==============================] - 184s 1s/step - loss: 2.5319e-05 - accuracy: 1.0000 - val_loss: 4.5076e-06 - val_accuracy: 1.0000
Epoch 7/10
158/158 [==============================] - 185s 1s/step - loss: 1.9726e-05 - accuracy

In [12]:
#Evaluate
predProbs = model.predict(testX)
predIdxs = (predProbs > 0.5).astype("int32")

print(classification_report(testY, predIdxs, target_names=lb.classes_))

40/40 [==============================] - 24s 535ms/step
                precision    recall  f1-score   support

Thermal_images       1.00      1.00      1.00      1268

      accuracy                           1.00      1268
     macro avg       1.00      1.00      1.00      1268
  weighted avg       1.00      1.00      1.00      1268



In [13]:
model.save("hotspot_detector.h5")

C:\Users\khavy\anaconda3\envs\aiml\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
